In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("/usr/local/lib/python3.6/site-packages/libsvm/")
from svmutil import *
from subprocess import *
from scipy.spatial.distance import pdist, cdist, squareform

In [3]:
X_train = pd.read_csv('X_train.csv',header=None).values
Y_train = pd.read_csv('Y_train.csv',header=None).values.reshape(-1)
X_test = pd.read_csv('X_test.csv',header=None).values
Y_test = pd.read_csv('Y_test.csv',header=None).values.reshape(-1)

# Problem #1

In [12]:
kernels = ['linear','polynomial','RBF']
for i in range(3):
    print('Kernel: '+kernels[i])
    params = '-q -t '+ str(i)
    model = svm_train(Y_train,X_train,params)
    svm_predict(Y_test,X_test, model)

Kernel: linear
Accuracy = 95.08% (2377/2500) (classification)
Kernel: polynomial
Accuracy = 34.68% (867/2500) (classification)
Kernel: RBF
Accuracy = 95.32% (2383/2500) (classification)


# Problem #2

In [27]:
c_begin,c_end,c_step = (-5,5,1) # Cost
g_begin,g_end,g_step = (-5,5,1) # Gamma
d_begin,d_end,d_step = (2,4,1)  # degree(poly. kernel)
r_begin,r_end,r_step = (0,2,1)  # coef.(poly. kernel)
best_acc = [0,0,0]
best_params = [0,(0,0,0,0),(0,0)]
RBF_res = []
for c in range(c_begin,c_end+c_step,c_step):
    
    # Linear kernel
    params = '-q -v 5 -s 0 -t 0 -c {}'.format(2**c)
    acc = svm_train(Y_train,X_train,params)
    print('[Linear] {} {} (best: c={} acc={})'.format(c,acc,best_params[0],best_acc[0]))
    if acc > best_acc[0]:
        best_acc[0] = acc
        best_params[0] = 2**c
        
    # Polynomial kernel
    for g in range(g_begin,g_end+g_step,g_step):
        for d in range(d_begin,d_end+d_step,d_step):
            for r in range(r_begin,r_end+r_step,r_step):
                params = '-q -v 5 -s 0 -t 1 -c {} -g {} -d {} -r {}'.format(2**c,2**g,d,r)
                acc = svm_train(Y_train,X_train,params)
                print('[Polynomial] {} {} {} {} {} (best: c={} g={} d={} r={} acc={})'.format(c,g,d,r,acc,best_params[1][0],best_params[1][1],best_params[1][2],best_params[1][3],best_acc[1]))
                if acc > best_acc[1]:
                    best_acc[1] = acc
                    best_params[1] = (2**c,2**g,d,r)
        
    # RBF kernel
    for g in range(g_begin,g_end+g_step,g_step):
        params = '-q -v 5 -s 0 -t 1 -c {} -g {}'.format(2**c,2**g)
        acc = svm_train(Y_train,X_train,params)
        print('[RBF] {} {} {} (best: c={} g={} acc={})'.format(c,g,acc,best_params[2][0],best_params[2][1],best_acc[2]))
        if acc > best_acc[2]:
            best_acc[2] = acc
            best_params[2] = (2**c,2**g) 
        RBF_res.append((c,g,acc))
        
print("Best parameters of linear, polynomial, RBF kernel respectively: \n{}".format(best_params))
print("Best accuracy of linear, polynomial, RBF kernel respectively: \n{}".format(best_acc))

Cross Validation Accuracy = 97.16%
[Linear] -5 97.16 (best: c=0 acc=0)
Cross Validation Accuracy = 96.72%
[Polynomial] -5 -5 2 0 96.72 (best: c=0 g=0 d=0 r=0 acc=0)
Cross Validation Accuracy = 97.06%
[Polynomial] -5 -5 2 1 97.06 (best: c=0.03125 g=0.03125 d=2 r=0 acc=96.72)
Cross Validation Accuracy = 97.14%
[Polynomial] -5 -5 2 2 97.14 (best: c=0.03125 g=0.03125 d=2 r=1 acc=97.06)
Cross Validation Accuracy = 96.62%
[Polynomial] -5 -5 3 0 96.61999999999999 (best: c=0.03125 g=0.03125 d=2 r=2 acc=97.14)
Cross Validation Accuracy = 97.84%
[Polynomial] -5 -5 3 1 97.84 (best: c=0.03125 g=0.03125 d=2 r=2 acc=97.14)
Cross Validation Accuracy = 97.94%
[Polynomial] -5 -5 3 2 97.94 (best: c=0.03125 g=0.03125 d=3 r=1 acc=97.84)
Cross Validation Accuracy = 95.4%
[Polynomial] -5 -5 4 0 95.39999999999999 (best: c=0.03125 g=0.03125 d=3 r=2 acc=97.94)
Cross Validation Accuracy = 97.82%
[Polynomial] -5 -5 4 1 97.82 (best: c=0.03125 g=0.03125 d=3 r=2 acc=97.94)
Cross Validation Accuracy = 98.12%
[Polyno

Cross Validation Accuracy = 96.88%
[Polynomial] -5 2 4 2 96.88 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 98.04%
[Polynomial] -5 3 2 0 98.04 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 98.22%
[Polynomial] -5 3 2 1 98.22 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 98.24%
[Polynomial] -5 3 2 2 98.24000000000001 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 97.66%
[Polynomial] -5 3 3 0 97.66 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 97.8%
[Polynomial] -5 3 3 1 97.8 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 97.74%
[Polynomial] -5 3 3 2 97.74000000000001 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 96.44%
[Polynomial] -5 3 4 0 96.44 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 96.78%
[Polyno

Cross Validation Accuracy = 97.82%
[Polynomial] -4 -2 3 2 97.82 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 96.54%
[Polynomial] -4 -2 4 0 96.54 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 97.22%
[Polynomial] -4 -2 4 1 97.22 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 97.66%
[Polynomial] -4 -2 4 2 97.66 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 98.12%
[Polynomial] -4 -1 2 0 98.11999999999999 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 98.04%
[Polynomial] -4 -1 2 1 98.04 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 97.9%
[Polynomial] -4 -1 2 2 97.89999999999999 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 97.72%
[Polynomial] -4 -1 3 0 97.72 (best: c=0.03125 g=0.5 d=2 r=2 acc=98.24000000000001)
Cross Validation Accuracy = 97.7%

Cross Validation Accuracy = 96.86%
[Linear] -3 96.86 (best: c=0.03125 acc=97.16)
Cross Validation Accuracy = 97.64%
[Polynomial] -3 -5 2 0 97.64 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 97.7%
[Polynomial] -3 -5 2 1 97.7 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 97.8%
[Polynomial] -3 -5 2 2 97.8 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 97.5%
[Polynomial] -3 -5 3 0 97.5 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 98.08%
[Polynomial] -3 -5 3 1 98.08 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 97.8%
[Polynomial] -3 -5 3 2 97.8 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 96.38%
[Polynomial] -3 -5 4 0 96.38 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 98%
[Polynomial] -3 -5 4 1 98.0 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 98.08%
[Polynomial] -3 -5 4 2 98.08 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cros

Cross Validation Accuracy = 96.52%
[Polynomial] -3 3 4 0 96.52 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 96.62%
[Polynomial] -3 3 4 1 96.61999999999999 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 96.54%
[Polynomial] -3 3 4 2 96.54 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 97.98%
[Polynomial] -3 4 2 0 97.98 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 98.26%
[Polynomial] -3 4 2 1 98.26 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 98.24%
[Polynomial] -3 4 2 2 98.24000000000001 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 97.7%
[Polynomial] -3 4 3 0 97.7 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 97.46%
[Polynomial] -3 4 3 1 97.46000000000001 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 97.78%
[Polynomial] -3 4 3 2 97.78 (best: c=0.0625 g=2 d=2 r=0 acc=98.28)
Cross Validation Accuracy = 96.62%
[Polynomial] 

...

Cross Validation Accuracy = 96.54%
[Polynomial] 2 3 4 1 96.54 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 96.72%
[Polynomial] 2 3 4 2 96.72 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 98.06%
[Polynomial] 2 4 2 0 98.06 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.98%
[Polynomial] 2 4 2 1 97.98 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 98.06%
[Polynomial] 2 4 2 2 98.06 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.64%
[Polynomial] 2 4 3 0 97.64 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.66%
[Polynomial] 2 4 3 1 97.66 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.54%
[Polynomial] 2 4 3 2 97.54 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 96.52%
[Polynomial] 2 4 4 0 96.52 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 96.7%
[Polynomial] 2 4 4 1 96.7 (bes

Cross Validation Accuracy = 97.8%
[Polynomial] 3 0 3 0 97.8 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.46%
[Polynomial] 3 0 3 1 97.46000000000001 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.72%
[Polynomial] 3 0 3 2 97.72 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 96.54%
[Polynomial] 3 0 4 0 96.54 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 96.98%
[Polynomial] 3 0 4 1 96.98 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 96.98%
[Polynomial] 3 0 4 2 96.98 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 98.2%
[Polynomial] 3 1 2 0 98.2 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 98.06%
[Polynomial] 3 1 2 1 98.06 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.9%
[Polynomial] 3 1 2 2 97.89999999999999 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.52%
[Polynomia

Cross Validation Accuracy = 98.02%
[Polynomial] 4 -4 4 2 98.02 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 98%
[Polynomial] 4 -3 2 0 98.0 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 98.22%
[Polynomial] 4 -3 2 1 98.22 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 98.02%
[Polynomial] 4 -3 2 2 98.02 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.64%
[Polynomial] 4 -3 3 0 97.64 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.98%
[Polynomial] 4 -3 3 1 97.98 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 98.08%
[Polynomial] 4 -3 3 2 98.08 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 96.42%
[Polynomial] 4 -3 4 0 96.41999999999999 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.5%
[Polynomial] 4 -3 4 1 97.5 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.84%
[Polynomial] 4

Cross Validation Accuracy = 97.6%
[Polynomial] 4 5 3 2 97.6 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 96.56%
[Polynomial] 4 5 4 0 96.56 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 96.62%
[Polynomial] 4 5 4 1 96.61999999999999 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 96.66%
[Polynomial] 4 5 4 2 96.66 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.42%
[RBF] 4 -5 97.42 (best: c=0.5 g=0.03125 acc=97.88)
Cross Validation Accuracy = 97.7%
[RBF] 4 -4 97.7 (best: c=0.5 g=0.03125 acc=97.88)
Cross Validation Accuracy = 97.74%
[RBF] 4 -3 97.74000000000001 (best: c=0.5 g=0.03125 acc=97.88)
Cross Validation Accuracy = 97.7%
[RBF] 4 -2 97.7 (best: c=0.5 g=0.03125 acc=97.88)
Cross Validation Accuracy = 97.54%
[RBF] 4 -1 97.54 (best: c=0.5 g=0.03125 acc=97.88)
Cross Validation Accuracy = 97.76%
[RBF] 4 0 97.76 (best: c=0.5 g=0.03125 acc=97.88)
Cross Validation Accuracy = 97.54%
[RBF] 4 1 97.54 

Cross Validation Accuracy = 98.1%
[Polynomial] 5 2 2 1 98.1 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 98.06%
[Polynomial] 5 2 2 2 98.06 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.56%
[Polynomial] 5 2 3 0 97.56 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.78%
[Polynomial] 5 2 3 1 97.78 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 97.78%
[Polynomial] 5 2 3 2 97.78 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 96.5%
[Polynomial] 5 2 4 0 96.5 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 96.5%
[Polynomial] 5 2 4 1 96.5 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 96.56%
[Polynomial] 5 2 4 2 96.56 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 98.22%
[Polynomial] 5 3 2 0 98.22 (best: c=0.5 g=0.03125 d=3 r=1 acc=98.34)
Cross Validation Accuracy = 98.04%
[Polynomial] 5 3 2 1 98.04 (best: c

In [30]:
begin_level = round(max(x[2] for x in RBF_res)) - 3
step_size = 0.5
gnuplot = Popen('/usr/local/bin/gnuplot',stdin = PIPE,stdout=PIPE,stderr=PIPE).stdin
gnuplot.write(b"set term png transparent small linewidth 2 medium enhanced\n")
gnuplot.write("set output \"OUT.png\"\n".encode())
gnuplot.write(b"set xlabel \"log2(C)\"\n")
gnuplot.write(b"set ylabel \"log2(gamma)\"\n")
gnuplot.write("set xrange [{0}:{1}]\n".format(c_begin,c_end).encode())
gnuplot.write("set yrange [{0}:{1}]\n".format(g_begin,g_end).encode())
gnuplot.write(b"set contour\n")
gnuplot.write("set cntrparam levels incremental {0},{1},100\n".format(begin_level,step_size).encode())
gnuplot.write(b"unset surface\n")
gnuplot.write(b"unset ztics\n")
gnuplot.write(b"set view 0,0\n")
gnuplot.write("set title \"HW5 - handwritten digit\"\n".encode())
gnuplot.write(b"unset label\n")
gnuplot.write("set label \"Best log2(C) = {0}  log2(gamma) = {1}  accuracy = {2}%\" \
  at screen 0.5,0.85 center\n". \
  format(np.log2(best_params[2][0]), np.log2(best_params[2][1]), best_acc[2]).encode())
gnuplot.write("set label \"C = {0}  gamma = {1}\""
  " at screen 0.5,0.8 center\n".format(best_params[2][0], best_params[2][1]).encode())
gnuplot.write(b"set key at screen 0.9,0.95\n")
gnuplot.write(b"splot \"-\" with lines\n")

RBF_res.sort(key = lambda x:(x[0], -x[1]))

prevc = RBF_res[0][0]
for line in RBF_res:
    if prevc != line[0]:
        gnuplot.write(b"\n")
        prevc = line[0]
    gnuplot.write("{0[0]} {0[1]} {0[2]}\n".format(line).encode())
gnuplot.write(b"e\n")
gnuplot.write(b"\n") # force gnuplot back to prompt when term set failure
gnuplot.flush()

In [31]:
best_params

[0.03125, (0.5, 0.03125, 3, 1), (0.5, 0.03125)]

In [32]:
best_acc

[97.16, 98.34, 97.88]

In [33]:
# Best Linear
params = '-q -t 0 -c {}'.format(best_params[0])
model = svm_train(Y_train,X_train,params)
svm_predict(Y_test,X_test,model)

# Best Poly
params = '-q -t 1 -c {} -g {} -d {} -r {}'.format(best_params[1][0],best_params[1][1],best_params[1][2],best_params[1][3])
model = svm_train(Y_train,X_train,params)
svm_predict(Y_test,X_test,model)

# Best RBF
params = '-q -t 2 -c {} -g {}'.format(best_params[2][0],best_params[2][1])
model = svm_train(Y_train,X_train,params)
svm_predict(Y_test,X_test,model)

Accuracy = 96% (2400/2500) (classification)
Accuracy = 97.92% (2448/2500) (classification)
Accuracy = 98.04% (2451/2500) (classification)


([1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  3.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  3.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,

# problem #3

In [18]:
gamma = -0.03125

linear_train = np.dot(X_train,X_train.T)
RBF_train = squareform(np.exp(gamma*pdist(X_train,'sqeuclidean')))
x_train_kernel = np.hstack((np.arange(1, 5001).reshape((5000, 1)),np.add(linear_train,RBF_train)))

linear_test = np.dot(X_test,X_train.T)
RBF_test = np.exp(gamma*cdist(X_test,X_train,'sqeuclidean'))
x_test_kernel = np.hstack((np.arange(1, 2501).reshape((2500, 1)),np.add(linear_test,RBF_test)))

In [19]:
model = svm_train(Y_train,x_train_kernel,'-q -t 4')

In [20]:
svm_predict(Y_test,x_test_kernel,model)

Accuracy = 95.32% (2383/2500) (classification)


([1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  3.0,
  1.0,
  1.0,
  1.0,

In [25]:
np.log2(0.03125)

-5.0